# Интеллектуальная обработка данных

Подключение нужных инструментов

In [8]:
#основные библиотеки
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
#ML библиотеки
from sklearn.model_selection import train_test_split
import cv2
#библиотека для обработки аудио
import tensorflow
import librosa
import librosa.display
import IPython.display as ipd
#дополнитеьлные инструменты
from tqdm import tqdm
import pylab

### Описание датасета

In [17]:
DIRS = os.listdir('./data/')
DIRS

['blues',
 'classical',
 'country',
 'disco',
 'hiphop',
 'jazz',
 'metal',
 'pop',
 'reggae',
 'rock']

### Визуализация аудиоинформации в mel-спектрограммы

In [39]:
def read_wav_file(path, preprocess=True): 
    """Reads wav file and cuts it to 3 10-seconds long parts
    
    path: path to file
    preprocess: if True, apply: y(t) = x(t) − α ∗ x(t−1);
    where α is set to 0.97 in order to boost higher frequencies
    
    Returns: list of signals as np.arrays and sample rate as int
    """
    y, sr = librosa.load(path)
    if preprocess:
        x = y.copy()
        z = y[1:] - 0.97 * x[:-1]
        y = z
        
    signals = []
    for signal_id in range(3):
        signals.append(z[signal_id*sr*10:(signal_id+1)*sr*10])
    return signals, sr


def create_spectrogram(z, sr, save_path):
    """Saves signal as MEL-spectrogram
    
    z: signal (np.array)
    sr: sample rate (usually 22050)
    save_path: MEL-spectrogram save path
    """
    #matplotlib.use('Agg')    #uncomment this line when in *.py file extension
    pylab.axis('off')
    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
    S = librosa.feature.melspectrogram(y=z, sr=sr)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    pylab.savefig(save_path, bbox_inches=None, pad_inches=0)
    pylab.close()
    
    
def resize_image(path):    
    """
    Loads image, resizes and saves it to the same path
    """
    SIZE = 224, 224
    im = Image.open(path)
    resized_im = im.resize(SIZE)
    resized_im.save(path)

def full_preprocess(path, save_path):
    """
    Full preprocess of audio file
    """
    y, sr = read_wav_file(path)
    idx = 0
    for signal_id in y:
        save_to = save_path + '_' + str(idx) + '.jpg'
        create_spectrogram(signal_id, sr, save_to)
        resize_image(save_to)
        idx += 1
        
def prepare_dataset(path='./'):
    """
    Creates spectograms for all audio files in dataset and separates them into
    'train' and 'test' folders
    """
    if 'mels' not in os.listdir():
        os.mkdir('./' + 'mels')
    
    genres = os.listdir(path + '/data')
    for genre in genres:
        if genre not in ('./' + 'mels'):
            os.mkdir('./' + 'mels' + '/' + genre)
        audios = os.listdir(path + '/data/' + genre)
        audios_amount = len(audios)
        id_audio = 0
        for id_audio in tqdm(range(audios_amount)):
            folder = './mels/'
            full_preprocess(path + '/data/' + genre + '/' + audios[id_audio], \
                            folder + genre + '/' + audios[id_audio][:-3] + '.jpg')

In [40]:
prepare_dataset()

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:59<00:00,  1.67it/s]


In [41]:
# проинициализируем два массива, один для изображений, второй для названий
images_arr = []
labels_arr = []

In [42]:
def upload_from_dir(flower_type, DIR):
    for img in os.listdir(DIR):
        label = flower_type
        path = os.path.join(DIR, img)
        img = cv2.imread(path, cv2.IMREAD_COLOR)      
        images_arr.append(np.array(img))
        labels_arr.append(str(label))

In [43]:
for DIR in tqdm(DIRS):
    upload_from_dir(DIR, './mels/' + DIR)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:29<00:00,  2.98s/it]


### Нормализация спектрограмм с учетом особенностей человеческого уха

### Разделение даты на обучающую и тестовую выборки

### Настройка нейронной сети

### Обучение 

### Изменение гиперпараметров сети

### Проверка сети на тестовой выборке

### Анализ реузльтатов